In [ ]:
import os

os.chdir("../..")

In [ ]:
%pip install umap-learn[plot]

In [ ]:
import torch
import umap

In [ ]:
model = torch.load("path/to/your/model.pth")

tokenizer = ...  # Initialize tokenizer

embeddings = model.embed.weight.detach().cpu().numpy()

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
embedding_2d = reducer.fit_transform(embeddings)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], s=10, alpha=0.7)

for i in range(0, len(embedding_2d), max(1, len(embedding_2d)//200)):  # only sample some labels
    plt.text(embedding_2d[i, 0], embedding_2d[i, 1],
                tokenizer.inv_vocab[i],
                fontsize=8, alpha=0.7)

plt.title("UMAP projection of SketchTransformer token embeddings")
plt.show()